In [7]:
import tentaclio as tio
import pandas as pd

In [8]:
def create_or_update_aws_table(
    tentaclio_db,
    database,
    table,
    s3_location,
    query,
    create_table_first=False,
    partition_col=None,
    with_no_data = False
):
    
    if with_no_data == False:
        with_no_data_str = ''
    else:
        with_no_data_str = 'WITH NO DATA'

    if partition_col is None:
        add_partition_str = ''
    else:
        add_partition_str = f",partitioned_by = ARRAY['{partition_col}']"        
        
    if create_table_first == True:    
        
        drop_query = f'''
            DROP TABLE IF EXISTS {database}.{table}
        '''
        with tentaclio_db:
            pd.read_sql_query(drop_query, tentaclio_db.conn)
        
        create_query = f'''
        CREATE TABLE {database}.{table} WITH (
        format='parquet'
        ,external_location='{s3_location}'
        {add_partition_str}
         )
        AS (
        {query}
        )
        {with_no_data_str};
        '''
        with tentaclio_db:
            pd.read_sql_query(create_query, tentaclio_db.conn)

            
    update_query = f'''
        INSERT INTO {database}.{table}
        {query};
        '''
    with tentaclio_db:
        pd.read_sql_query(update_query, tentaclio_db.conn)    
        
    return None

In [9]:
testing_query = f'''
SELECT
    dt.datetime_utc,
    SUM(
        MAX_BY(period_profile_coefficient_value, (execution_date, ap_map.priority, pt_map.priority)))
        OVER (partition by ig.industry_group_id
             ORDER BY  ig.industry_group_id, dt.datetime_utc rows unbounded preceding
    ) AS cumulative_period_profile_coefficient_value,
    ig.industry_group_id
    FROM "settlement"."ppc" ppc
    LEFT JOIN "settlement".actual_predicted_priority_map ap_map
    ON ap_map.actual_predicted_indicator = ppc.actual_predicted_indicator
    LEFT JOIN "settlement".prediction_type_priority_map pt_map
    ON pt_map.daily_monthly_flag = ppc.daily_monthly_flag
    LEFT JOIN settlement.datetime_lookup dt
    ON dt.settlement_date = ppc.settlement_date
    AND dt.settlement_period = ppc.settlement_period_id
    LEFT JOIN volumes.industry_group ig
    ON ig.gsp_group_id = ppc.gsp_group_id
    AND ig.profile_class_id = ppc.profile_class_id
    AND ig.standard_settlement_configuration_id = ppc.standard_settlement_configuration_id
    AND ig.time_pattern_regime_id = ppc.time_pattern_regime_id
    WHERE ig.industry_group_id = 1
    AND ig.industry_group_id = 1
    GROUP BY 1, 3
    ORDER BY 1, 3
'''

create_or_update_aws_table(
    tio.db('awsathena+rest://hostname/settlement'),
    'volumes',
    'testing',
    's3://octoenergy-production-pipelines/Store/Volumes/testing/',
    testing_query,
    create_table_first=True,
    partition_col='industry_group_id',
    with_no_data = True
)

Failed to download csv.
Traceback (most recent call last):
  File "/Users/josheg/.pyenv/versions/3.6.8/envs/data-analysis/lib/python3.6/site-packages/pyathena/result_set.py", line 422, in _as_pandas
    Key=key)
  File "/Users/josheg/.pyenv/versions/3.6.8/envs/data-analysis/lib/python3.6/site-packages/pyathena/util.py", line 57, in retry_api_call
    return retry(func, *args, **kwargs)
  File "/Users/josheg/.pyenv/versions/3.6.8/envs/data-analysis/lib/python3.6/site-packages/tenacity/__init__.py", line 358, in call
    do = self.iter(retry_state=retry_state)
  File "/Users/josheg/.pyenv/versions/3.6.8/envs/data-analysis/lib/python3.6/site-packages/tenacity/__init__.py", line 319, in iter
    return fut.result()
  File "/Users/josheg/.pyenv/versions/3.6.8/lib/python3.6/concurrent/futures/_base.py", line 425, in result
    return self.__get_result()
  File "/Users/josheg/.pyenv/versions/3.6.8/lib/python3.6/concurrent/futures/_base.py", line 384, in __get_result
    raise self._exception


OperationalError: (pyathena.error.OperationalError) An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.
[SQL: 
        CREATE TABLE volumes.testing WITH (
        format='parquet'
        ,external_location='s3://octoenergy-production-pipelines/Store/Volumes/testing/'
        ,partitioned_by = ARRAY['industry_group_id']
         )
        AS (
        
SELECT
    dt.datetime_utc,
    SUM(
        MAX_BY(period_profile_coefficient_value, (execution_date, ap_map.priority, pt_map.priority)))
        OVER (partition by ig.industry_group_id
             ORDER BY  ig.industry_group_id, dt.datetime_utc rows unbounded preceding
    ) AS cumulative_period_profile_coefficient_value,
    ig.industry_group_id
    FROM "settlement"."ppc" ppc
    LEFT JOIN "settlement".actual_predicted_priority_map ap_map
    ON ap_map.actual_predicted_indicator = ppc.actual_predicted_indicator
    LEFT JOIN "settlement".prediction_type_priority_map pt_map
    ON pt_